# Wellbeing and Lifestyle Data Analysis

## Section 1: Exploratory Data Analysis (EDA)

This notebook performs comprehensive exploratory data analysis on the wellbeing and lifestyle dataset, including:
- Data quality checks
- Correlation analysis
- Distribution visualizations
- Outlier detection and handling
- Summary statistics by demographic groups


In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
warnings.filterwarnings('ignore')

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("Libraries imported successfully!")


## 1.1 Load Dataset and Check Data Quality


In [ ]:
# Load the dataset
df = pd.read_csv('Wellbeing_and_lifestyle_data_Kaggle.csv')

# Convert numeric columns to proper dtypes
# All columns except Timestamp, AGE, and GENDER should be numeric
numeric_columns = [col for col in df.columns if col not in ['Timestamp', 'AGE', 'GENDER']]
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Display basic information
print("=" * 80)
print("DATASET SHAPE")
print("=" * 80)
print(f"Number of rows: {df.shape[0]}")
print(f"Number of columns: {df.shape[1]}")
print("\n")

# Display first few rows
print("=" * 80)
print("FIRST 5 ROWS")
print("=" * 80)
df.head()


In [ ]:
# Data types and basic information
print("=" * 80)
print("DATASET INFO")
print("=" * 80)
df.info()
print("\n")

print("=" * 80)
print("COLUMN NAMES")
print("=" * 80)
print(df.columns.tolist())


In [ ]:
# Check for missing values
print("=" * 80)
print("MISSING VALUES")
print("=" * 80)

missing_values = df.isnull().sum()
missing_percent = (df.isnull().sum() / len(df)) * 100
missing_df = pd.DataFrame({
    'Column': missing_values.index,
    'Missing_Count': missing_values.values,
    'Missing_Percent': missing_percent.values
})
missing_df = missing_df[missing_df['Missing_Count'] > 0].sort_values('Missing_Count', ascending=False)

if len(missing_df) > 0:
    print(missing_df.to_string(index=False))
else:
    print("No missing values found in the dataset!")
    
print(f"\nTotal missing values: {df.isnull().sum().sum()}")


In [ ]:
# Check for duplicate rows
print("=" * 80)
print("DUPLICATE ROWS")
print("=" * 80)
duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")
print(f"Percentage of duplicates: {(duplicates/len(df))*100:.2f}%")


In [ ]:
# Basic descriptive statistics
print("=" * 80)
print("DESCRIPTIVE STATISTICS FOR NUMERICAL FEATURES")
print("=" * 80)
df.describe()


In [ ]:
# Check categorical variables
print("=" * 80)
print("CATEGORICAL VARIABLES DISTRIBUTION")
print("=" * 80)

categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
print(f"Categorical columns: {categorical_cols}\n")

for col in categorical_cols:
    if col != 'Timestamp':  # Skip timestamp
        print(f"\n{col}:")
        print(df[col].value_counts())
        print(f"Unique values: {df[col].nunique()}")


## 1.2 Correlation Analysis


In [ ]:
# Select only numerical columns for correlation analysis
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
print(f"Numerical columns for correlation: {len(numerical_cols)}")
print(numerical_cols)


In [ ]:
# Calculate correlation matrix
correlation_matrix = df[numerical_cols].corr()

# Create a large figure for the correlation heatmap
plt.figure(figsize=(20, 16))
sns.heatmap(correlation_matrix, 
            annot=True, 
            fmt='.2f', 
            cmap='coolwarm', 
            center=0,
            square=True,
            linewidths=0.5,
            cbar_kws={"shrink": 0.8})
plt.title('Correlation Heatmap of All Numerical Features', fontsize=16, fontweight='bold', pad=20)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

print("\nCorrelation Matrix Shape:", correlation_matrix.shape)


In [ ]:
# Find strongest correlations with WORK_LIFE_BALANCE_SCORE (target variable)
print("=" * 80)
print("TOP CORRELATIONS WITH WORK_LIFE_BALANCE_SCORE")
print("=" * 80)

target_correlations = correlation_matrix['WORK_LIFE_BALANCE_SCORE'].sort_values(ascending=False)
print("\nTop positive correlations:")
print(target_correlations.head(10))

print("\nTop negative correlations:")
print(target_correlations.tail(10))


In [ ]:
# Visualize top correlations with target variable
top_features = target_correlations.abs().sort_values(ascending=False)[1:16]  # Exclude itself, take top 15

plt.figure(figsize=(12, 8))
top_features.sort_values().plot(kind='barh', color=['red' if x < 0 else 'green' for x in top_features.sort_values()])
plt.title('Top 15 Features Correlated with Work-Life Balance Score', fontsize=14, fontweight='bold')
plt.xlabel('Correlation Coefficient', fontsize=12)
plt.ylabel('Features', fontsize=12)
plt.axvline(x=0, color='black', linestyle='--', linewidth=0.8)
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()


## 1.3 Distribution Visualizations


In [ ]:
# Distribution of the target variable (WORK_LIFE_BALANCE_SCORE)
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histogram
axes[0].hist(df['WORK_LIFE_BALANCE_SCORE'], bins=50, color='skyblue', edgecolor='black', alpha=0.7)
axes[0].set_title('Distribution of Work-Life Balance Score', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Work-Life Balance Score', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].axvline(df['WORK_LIFE_BALANCE_SCORE'].mean(), color='red', linestyle='--', 
                linewidth=2, label=f'Mean: {df["WORK_LIFE_BALANCE_SCORE"].mean():.2f}')
axes[0].axvline(df['WORK_LIFE_BALANCE_SCORE'].median(), color='green', linestyle='--', 
                linewidth=2, label=f'Median: {df["WORK_LIFE_BALANCE_SCORE"].median():.2f}')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Box plot
axes[1].boxplot(df['WORK_LIFE_BALANCE_SCORE'], vert=True, patch_artist=True,
                boxprops=dict(facecolor='lightblue', color='blue'),
                medianprops=dict(color='red', linewidth=2),
                whiskerprops=dict(color='blue'),
                capprops=dict(color='blue'))
axes[1].set_title('Box Plot of Work-Life Balance Score', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Work-Life Balance Score', fontsize=12)
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Print statistics
print(f"Mean: {df['WORK_LIFE_BALANCE_SCORE'].mean():.2f}")
print(f"Median: {df['WORK_LIFE_BALANCE_SCORE'].median():.2f}")
print(f"Std Dev: {df['WORK_LIFE_BALANCE_SCORE'].std():.2f}")
print(f"Min: {df['WORK_LIFE_BALANCE_SCORE'].min():.2f}")
print(f"Max: {df['WORK_LIFE_BALANCE_SCORE'].max():.2f}")


In [ ]:
# Create histograms for all numerical features
# Exclude Timestamp if present in numerical columns
plot_cols = [col for col in numerical_cols if col != 'Timestamp']

# Calculate grid dimensions
n_cols = 4
n_rows = int(np.ceil(len(plot_cols) / n_cols))

fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, n_rows * 4))
axes = axes.flatten()

for idx, col in enumerate(plot_cols):
    axes[idx].hist(df[col].dropna(), bins=30, color='steelblue', edgecolor='black', alpha=0.7)
    axes[idx].set_title(f'{col}', fontsize=10, fontweight='bold')
    axes[idx].set_xlabel(col, fontsize=9)
    axes[idx].set_ylabel('Frequency', fontsize=9)
    axes[idx].grid(alpha=0.3)
    
    # Add mean line
    mean_val = df[col].mean()
    axes[idx].axvline(mean_val, color='red', linestyle='--', linewidth=1.5, alpha=0.7)

# Hide extra subplots
for idx in range(len(plot_cols), len(axes)):
    axes[idx].axis('off')

plt.suptitle('Distribution of All Numerical Features', fontsize=16, fontweight='bold', y=1.002)
plt.tight_layout()
plt.show()


In [ ]:
# Create box plots for key features to visualize spread and outliers
key_features = ['FRUITS_VEGGIES', 'DAILY_STRESS', 'PLACES_VISITED', 'CORE_CIRCLE', 
                'SOCIAL_NETWORK', 'ACHIEVEMENT', 'BMI_RANGE', 'TODO_COMPLETED',
                'FLOW', 'DAILY_STEPS', 'SLEEP_HOURS', 'DAILY_SHOUTING',
                'TIME_FOR_PASSION', 'WEEKLY_MEDITATION', 'WORK_LIFE_BALANCE_SCORE']

n_cols = 3
n_rows = int(np.ceil(len(key_features) / n_cols))

fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, n_rows * 4))
axes = axes.flatten()

for idx, col in enumerate(key_features):
    bp = axes[idx].boxplot(df[col].dropna(), vert=True, patch_artist=True,
                           boxprops=dict(facecolor='lightcoral', color='darkred', alpha=0.7),
                           medianprops=dict(color='blue', linewidth=2),
                           whiskerprops=dict(color='darkred'),
                           capprops=dict(color='darkred'),
                           flierprops=dict(marker='o', markerfacecolor='red', markersize=4, alpha=0.5))
    axes[idx].set_title(f'{col}', fontsize=10, fontweight='bold')
    axes[idx].set_ylabel('Value', fontsize=9)
    axes[idx].grid(alpha=0.3, axis='y')

# Hide extra subplots
for idx in range(len(key_features), len(axes)):
    axes[idx].axis('off')

plt.suptitle('Box Plots of Key Features (Outlier Detection)', fontsize=16, fontweight='bold', y=1.002)
plt.tight_layout()
plt.show()


## 1.4 Outlier Detection and Handling


In [ ]:
# Function to detect outliers using IQR method
def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

# Detect outliers for all numerical columns
print("=" * 80)
print("OUTLIER DETECTION (IQR METHOD)")
print("=" * 80)

outlier_summary = []

for col in plot_cols:
    outliers, lower, upper = detect_outliers_iqr(df, col)
    outlier_count = len(outliers)
    outlier_percent = (outlier_count / len(df)) * 100
    
    outlier_summary.append({
        'Feature': col,
        'Outlier_Count': outlier_count,
        'Outlier_Percent': outlier_percent,
        'Lower_Bound': lower,
        'Upper_Bound': upper
    })

outlier_df = pd.DataFrame(outlier_summary).sort_values('Outlier_Count', ascending=False)
print(outlier_df.to_string(index=False))


In [ ]:
# Visualize outliers
features_with_outliers = outlier_df[outlier_df['Outlier_Count'] > 0]['Feature'].head(6).tolist()

if len(features_with_outliers) > 0:
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    axes = axes.flatten()
    
    for idx, col in enumerate(features_with_outliers):
        if idx < len(axes):
            axes[idx].scatter(range(len(df)), df[col], alpha=0.5, s=10, color='steelblue')
            
            # Get bounds
            outliers, lower, upper = detect_outliers_iqr(df, col)
            axes[idx].axhline(y=lower, color='red', linestyle='--', linewidth=2, label=f'Lower: {lower:.2f}')
            axes[idx].axhline(y=upper, color='green', linestyle='--', linewidth=2, label=f'Upper: {upper:.2f}')
            
            axes[idx].set_title(f'{col}\n({len(outliers)} outliers, {(len(outliers)/len(df)*100):.2f}%)', 
                               fontsize=10, fontweight='bold')
            axes[idx].set_xlabel('Index', fontsize=9)
            axes[idx].set_ylabel('Value', fontsize=9)
            axes[idx].legend(fontsize=8)
            axes[idx].grid(alpha=0.3)
    
    # Hide extra subplots
    for idx in range(len(features_with_outliers), len(axes)):
        axes[idx].axis('off')
    
    plt.suptitle('Outlier Visualization for Top Features', fontsize=14, fontweight='bold', y=1.002)
    plt.tight_layout()
    plt.show()
else:
    print("No significant outliers detected.")


In [ ]:
# Outlier handling strategy
print("=" * 80)
print("OUTLIER HANDLING STRATEGY")
print("=" * 80)

# Create a copy for clean data
df_clean = df.copy()

# For this analysis, we'll use capping method (Winsorization) for extreme outliers
# This preserves the data while reducing the impact of extreme values

def cap_outliers(data, column, lower_percentile=0.01, upper_percentile=0.99):
    """Cap outliers at specified percentiles"""
    lower_cap = data[column].quantile(lower_percentile)
    upper_cap = data[column].quantile(upper_percentile)
    
    original_min = data[column].min()
    original_max = data[column].max()
    
    data[column] = data[column].clip(lower=lower_cap, upper=upper_cap)
    
    return data, lower_cap, upper_cap, original_min, original_max

# Apply capping to features with significant outliers (>5% outliers)
features_to_cap = outlier_df[outlier_df['Outlier_Percent'] > 5]['Feature'].tolist()

print(f"\nFeatures with >5% outliers to be capped: {len(features_to_cap)}")
print(features_to_cap)

capping_summary = []

for col in features_to_cap:
    df_clean, lower_cap, upper_cap, orig_min, orig_max = cap_outliers(df_clean, col)
    capping_summary.append({
        'Feature': col,
        'Original_Min': orig_min,
        'Original_Max': orig_max,
        'Capped_Lower': lower_cap,
        'Capped_Upper': upper_cap
    })

if len(capping_summary) > 0:
    print("\nCapping Summary:")
    print(pd.DataFrame(capping_summary).to_string(index=False))
else:
    print("\nNo features required capping (all features have <5% outliers).")

print(f"\nClean dataset shape: {df_clean.shape}")
print(f"Original dataset shape: {df.shape}")


## 1.5 Summary Statistics by Demographics


In [ ]:
# Ensure all numerical columns are properly typed
# Convert object columns to numeric where appropriate
print("=" * 80)
print("DATA TYPE VERIFICATION AND CONVERSION")
print("=" * 80)

print("\nData types before conversion:")
print(df_clean.dtypes)

# Convert all numerical feature columns to numeric type
for col in numerical_cols:
    if col in df_clean.columns:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

print("\n" + "=" * 80)
print("Data types after conversion:")
print(df_clean.dtypes)

# Check if any conversions resulted in NaN
print("\n" + "=" * 80)
print("Missing values after type conversion:")
print(df_clean.isnull().sum().sum())


In [ ]:
# Summary statistics by Gender
print("=" * 80)
print("SUMMARY STATISTICS BY GENDER")
print("=" * 80)

print("\nGender Distribution:")
print(df_clean['GENDER'].value_counts())
print(f"\nGender proportions:")
print(df_clean['GENDER'].value_counts(normalize=True) * 100)

print("\n" + "=" * 80)
print("WORK-LIFE BALANCE SCORE BY GENDER")
print("=" * 80)
gender_stats = df_clean.groupby('GENDER')['WORK_LIFE_BALANCE_SCORE'].agg([
    'count', 'mean', 'median', 'std', 'min', 'max'
]).round(2)
print(gender_stats)


In [ ]:
# Visualize Work-Life Balance Score by Gender
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Box plot
df_clean.boxplot(column='WORK_LIFE_BALANCE_SCORE', by='GENDER', ax=axes[0], patch_artist=True)
axes[0].set_title('Work-Life Balance Score by Gender', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Gender', fontsize=11)
axes[0].set_ylabel('Work-Life Balance Score', fontsize=11)
axes[0].grid(alpha=0.3)
plt.sca(axes[0])
plt.xticks(rotation=0)

# Violin plot
import seaborn as sns
sns.violinplot(data=df_clean, x='GENDER', y='WORK_LIFE_BALANCE_SCORE', ax=axes[1], palette='Set2')
axes[1].set_title('Distribution of Work-Life Balance Score by Gender', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Gender', fontsize=11)
axes[1].set_ylabel('Work-Life Balance Score', fontsize=11)
axes[1].grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.show()


In [ ]:
# Summary statistics by Age Group
print("=" * 80)
print("SUMMARY STATISTICS BY AGE GROUP")
print("=" * 80)

print("\nAge Group Distribution:")
print(df_clean['AGE'].value_counts().sort_index())
print(f"\nAge Group proportions:")
print(df_clean['AGE'].value_counts(normalize=True).sort_index() * 100)

print("\n" + "=" * 80)
print("WORK-LIFE BALANCE SCORE BY AGE GROUP")
print("=" * 80)
age_stats = df_clean.groupby('AGE')['WORK_LIFE_BALANCE_SCORE'].agg([
    'count', 'mean', 'median', 'std', 'min', 'max'
]).round(2)
print(age_stats)


In [ ]:
# Visualize Work-Life Balance Score by Age Group
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Define age order for proper sorting
age_order = ['Less than 20', '21 to 35', '36 to 50', '51 or more']

# Box plot
sns.boxplot(data=df_clean, x='AGE', y='WORK_LIFE_BALANCE_SCORE', ax=axes[0], 
            order=age_order, palette='viridis')
axes[0].set_title('Work-Life Balance Score by Age Group', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Age Group', fontsize=11)
axes[0].set_ylabel('Work-Life Balance Score', fontsize=11)
axes[0].grid(alpha=0.3, axis='y')
axes[0].tick_params(axis='x', rotation=45)

# Violin plot
sns.violinplot(data=df_clean, x='AGE', y='WORK_LIFE_BALANCE_SCORE', ax=axes[1], 
               order=age_order, palette='viridis')
axes[1].set_title('Distribution of Work-Life Balance Score by Age Group', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Age Group', fontsize=11)
axes[1].set_ylabel('Work-Life Balance Score', fontsize=11)
axes[1].grid(alpha=0.3, axis='y')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()


In [ ]:
# Combined Analysis: Gender x Age Group
print("=" * 80)
print("WORK-LIFE BALANCE SCORE BY GENDER AND AGE GROUP")
print("=" * 80)

# Create pivot table
pivot_stats = df_clean.groupby(['AGE', 'GENDER'])['WORK_LIFE_BALANCE_SCORE'].agg([
    'count', 'mean', 'std'
]).round(2)
print(pivot_stats)

# Visualize interaction
plt.figure(figsize=(12, 6))
for gender in df_clean['GENDER'].unique():
    gender_data = df_clean[df_clean['GENDER'] == gender]
    age_means = gender_data.groupby('AGE')['WORK_LIFE_BALANCE_SCORE'].mean().reindex(age_order)
    plt.plot(age_order, age_means, marker='o', linewidth=2, markersize=8, label=gender)

plt.title('Work-Life Balance Score by Age and Gender', fontsize=14, fontweight='bold')
plt.xlabel('Age Group', fontsize=12)
plt.ylabel('Mean Work-Life Balance Score', fontsize=12)
plt.legend(title='Gender', fontsize=11)
plt.grid(alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# Additional demographic insights - Key features by gender
print("=" * 80)
print("KEY LIFESTYLE FEATURES BY GENDER")
print("=" * 80)

lifestyle_features = ['FRUITS_VEGGIES', 'DAILY_STRESS', 'SLEEP_HOURS', 'DAILY_STEPS', 
                      'WEEKLY_MEDITATION', 'TIME_FOR_PASSION', 'FLOW']

# Ensure lifestyle features are numeric
for col in lifestyle_features:
    if col in df_clean.columns:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

gender_lifestyle = df_clean.groupby('GENDER')[lifestyle_features].mean().round(2)
print(gender_lifestyle.T)

# Visualize
fig, axes = plt.subplots(2, 4, figsize=(20, 10))
axes = axes.flatten()

for idx, feature in enumerate(lifestyle_features):
    sns.barplot(data=df_clean, x='GENDER', y=feature, ax=axes[idx], palette='Set2', ci=None)
    axes[idx].set_title(f'{feature}', fontsize=11, fontweight='bold')
    axes[idx].set_xlabel('Gender', fontsize=10)
    axes[idx].set_ylabel('Mean Value', fontsize=10)
    axes[idx].grid(alpha=0.3, axis='y')

# Hide last subplot
axes[7].axis('off')

plt.suptitle('Lifestyle Features by Gender', fontsize=16, fontweight='bold', y=1.002)
plt.tight_layout()
plt.show()


In [ ]:
# Key features by age group
print("=" * 80)
print("KEY LIFESTYLE FEATURES BY AGE GROUP")
print("=" * 80)

age_lifestyle = df_clean.groupby('AGE')[lifestyle_features].mean().round(2)
print(age_lifestyle)

# Visualize
fig, axes = plt.subplots(2, 4, figsize=(20, 10))
axes = axes.flatten()

for idx, feature in enumerate(lifestyle_features):
    sns.barplot(data=df_clean, x='AGE', y=feature, ax=axes[idx], 
                order=age_order, palette='viridis', ci=None)
    axes[idx].set_title(f'{feature}', fontsize=11, fontweight='bold')
    axes[idx].set_xlabel('Age Group', fontsize=10)
    axes[idx].set_ylabel('Mean Value', fontsize=10)
    axes[idx].tick_params(axis='x', rotation=45)
    axes[idx].grid(alpha=0.3, axis='y')

# Hide last subplot
axes[7].axis('off')

plt.suptitle('Lifestyle Features by Age Group', fontsize=16, fontweight='bold', y=1.002)
plt.tight_layout()
plt.show()


## 1.6 Save Clean Dataset


In [ ]:
# Save the clean dataset for future use
df_clean.to_csv('wellbeing_data_clean.csv', index=False)
print("Clean dataset saved as 'wellbeing_data_clean.csv'")
print(f"Shape: {df_clean.shape}")
print(f"Columns: {df_clean.columns.tolist()}")


# Section 2: Feature Engineering & Preprocessing

This section creates composite wellness scores and prepares the data for clustering analysis.


## 2.1 Create Composite Wellness Scores

We'll create three composite scores representing different dimensions of wellbeing:
- **Physical Health Score**: Combines nutrition, BMI, physical activity, and sleep
- **Mental Health Score**: Combines stress management, flow state, meditation, and vacation balance
- **Social Health Score**: Combines social connections and support networks


In [ ]:
# Import additional libraries for preprocessing
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import os

# Create a copy for feature engineering
df_features = df_clean.copy()

print("=" * 80)
print("STARTING FEATURE ENGINEERING")
print("=" * 80)
print(f"Starting dataset shape: {df_features.shape}")
print(f"Columns: {df_features.columns.tolist()}")


In [ ]:
# Function to normalize features to 0-10 scale for composite scores
def normalize_to_scale(series, target_min=0, target_max=10):
    """Normalize a series to a target scale (default 0-10)"""
    if series.max() == series.min():
        return series
    normalized = (series - series.min()) / (series.max() - series.min())
    return normalized * (target_max - target_min) + target_min

# 1. PHYSICAL HEALTH SCORE
print("=" * 80)
print("CREATING PHYSICAL HEALTH SCORE")
print("=" * 80)

# Components: FRUITS_VEGGIES, BMI_RANGE, DAILY_STEPS, SLEEP_HOURS
physical_components = {
    'FRUITS_VEGGIES': df_features['FRUITS_VEGGIES'],
    'BMI_RANGE': df_features['BMI_RANGE'],
    'DAILY_STEPS': df_features['DAILY_STEPS'],
    'SLEEP_HOURS': df_features['SLEEP_HOURS']
}

# Normalize each component to 0-10 scale
physical_normalized = {}
for name, component in physical_components.items():
    physical_normalized[name] = normalize_to_scale(component)
    print(f"{name}: min={component.min():.2f}, max={component.max():.2f}, mean={component.mean():.2f}")

# Create Physical Health Score (average of normalized components)
df_features['PHYSICAL_HEALTH_SCORE'] = (
    physical_normalized['FRUITS_VEGGIES'] + 
    physical_normalized['BMI_RANGE'] + 
    physical_normalized['DAILY_STEPS'] + 
    physical_normalized['SLEEP_HOURS']
) / 4

print(f"\nPhysical Health Score created!")
print(f"Mean: {df_features['PHYSICAL_HEALTH_SCORE'].mean():.2f}")
print(f"Std: {df_features['PHYSICAL_HEALTH_SCORE'].std():.2f}")
print(f"Range: [{df_features['PHYSICAL_HEALTH_SCORE'].min():.2f}, {df_features['PHYSICAL_HEALTH_SCORE'].max():.2f}]")


In [ ]:
# 2. MENTAL HEALTH SCORE
print("\n" + "=" * 80)
print("CREATING MENTAL HEALTH SCORE")
print("=" * 80)

# Components: DAILY_STRESS (inverted), FLOW, WEEKLY_MEDITATION, LOST_VACATION (inverted)
# Note: Lower stress and lower lost vacation are better, so we'll invert them

mental_components = {
    'DAILY_STRESS': df_features['DAILY_STRESS'],
    'FLOW': df_features['FLOW'],
    'WEEKLY_MEDITATION': df_features['WEEKLY_MEDITATION'],
    'LOST_VACATION': df_features['LOST_VACATION']
}

# Normalize each component
mental_normalized = {}
for name, component in mental_components.items():
    mental_normalized[name] = normalize_to_scale(component)
    print(f"{name}: min={component.min():.2f}, max={component.max():.2f}, mean={component.mean():.2f}")

# Invert DAILY_STRESS and LOST_VACATION (lower is better)
mental_normalized['DAILY_STRESS'] = 10 - mental_normalized['DAILY_STRESS']
mental_normalized['LOST_VACATION'] = 10 - mental_normalized['LOST_VACATION']

# Create Mental Health Score (average of normalized components)
df_features['MENTAL_HEALTH_SCORE'] = (
    mental_normalized['DAILY_STRESS'] + 
    mental_normalized['FLOW'] + 
    mental_normalized['WEEKLY_MEDITATION'] + 
    mental_normalized['LOST_VACATION']
) / 4

print(f"\nMental Health Score created!")
print(f"Mean: {df_features['MENTAL_HEALTH_SCORE'].mean():.2f}")
print(f"Std: {df_features['MENTAL_HEALTH_SCORE'].std():.2f}")
print(f"Range: [{df_features['MENTAL_HEALTH_SCORE'].min():.2f}, {df_features['MENTAL_HEALTH_SCORE'].max():.2f}]")


In [ ]:
# 3. SOCIAL HEALTH SCORE
print("\n" + "=" * 80)
print("CREATING SOCIAL HEALTH SCORE")
print("=" * 80)

# Components: CORE_CIRCLE, SOCIAL_NETWORK, SUPPORTING_OTHERS
social_components = {
    'CORE_CIRCLE': df_features['CORE_CIRCLE'],
    'SOCIAL_NETWORK': df_features['SOCIAL_NETWORK'],
    'SUPPORTING_OTHERS': df_features['SUPPORTING_OTHERS']
}

# Normalize each component
social_normalized = {}
for name, component in social_components.items():
    social_normalized[name] = normalize_to_scale(component)
    print(f"{name}: min={component.min():.2f}, max={component.max():.2f}, mean={component.mean():.2f}")

# Create Social Health Score (average of normalized components)
df_features['SOCIAL_HEALTH_SCORE'] = (
    social_normalized['CORE_CIRCLE'] + 
    social_normalized['SOCIAL_NETWORK'] + 
    social_normalized['SUPPORTING_OTHERS']
) / 3

print(f"\nSocial Health Score created!")
print(f"Mean: {df_features['SOCIAL_HEALTH_SCORE'].mean():.2f}")
print(f"Std: {df_features['SOCIAL_HEALTH_SCORE'].std():.2f}")
print(f"Range: [{df_features['SOCIAL_HEALTH_SCORE'].min():.2f}, {df_features['SOCIAL_HEALTH_SCORE'].max():.2f}]")


In [ ]:
# Visualize the composite scores
print("\n" + "=" * 80)
print("COMPOSITE SCORES SUMMARY")
print("=" * 80)

composite_scores = ['PHYSICAL_HEALTH_SCORE', 'MENTAL_HEALTH_SCORE', 'SOCIAL_HEALTH_SCORE']

# Check for NaN values in composite scores
print("\nChecking for missing values in composite scores:")
for score in composite_scores:
    nan_count = df_features[score].isna().sum()
    print(f"  {score}: {nan_count} NaN values")

# Fill NaN values with median for visualization (if any exist)
for score in composite_scores:
    if df_features[score].isna().sum() > 0:
        median_val = df_features[score].median()
        df_features[score] = df_features[score].fillna(median_val)
        print(f"  Filled {score} NaN with median: {median_val:.2f}")

summary_stats = df_features[composite_scores].describe().round(2)
print("\n")
print(summary_stats)

# Create visualizations
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# Histograms
for idx, score in enumerate(composite_scores):
    data = df_features[score].dropna()  # Drop NaN for safety
    axes[0, idx].hist(data, bins=30, color='skyblue', edgecolor='black', alpha=0.7)
    axes[0, idx].set_title(f'{score} Distribution', fontsize=11, fontweight='bold')
    axes[0, idx].set_xlabel(score, fontsize=10)
    axes[0, idx].set_ylabel('Frequency', fontsize=10)
    axes[0, idx].axvline(data.mean(), color='red', linestyle='--', 
                        linewidth=2, label=f'Mean: {data.mean():.2f}')
    axes[0, idx].legend()
    axes[0, idx].grid(alpha=0.3)

# Box plots
for idx, score in enumerate(composite_scores):
    data = df_features[score].dropna()  # Drop NaN for box plot
    axes[1, idx].boxplot(data, vert=True, patch_artist=True,
                        boxprops=dict(facecolor='lightgreen', color='darkgreen', alpha=0.7),
                        medianprops=dict(color='red', linewidth=2))
    axes[1, idx].set_title(f'{score} Box Plot', fontsize=11, fontweight='bold')
    axes[1, idx].set_ylabel('Score', fontsize=10)
    axes[1, idx].grid(alpha=0.3, axis='y')

plt.suptitle('Composite Wellness Scores', fontsize=16, fontweight='bold', y=1.002)
plt.tight_layout()
plt.show()


## 2.2 Encode Categorical Variables

We'll encode AGE and GENDER variables for machine learning models.


In [ ]:
print("=" * 80)
print("ENCODING CATEGORICAL VARIABLES")
print("=" * 80)

# Keep original categorical variables for reference
df_features['AGE_ORIGINAL'] = df_features['AGE']
df_features['GENDER_ORIGINAL'] = df_features['GENDER']

# 1. Label Encoding for AGE (ordinal variable)
age_order = ['Less than 20', '21 to 35', '36 to 50', '51 or more']
age_mapping = {age: idx for idx, age in enumerate(age_order)}

df_features['AGE_ENCODED'] = df_features['AGE'].map(age_mapping)
print("\nAge Encoding:")
for age, code in age_mapping.items():
    count = (df_features['AGE'] == age).sum()
    print(f"  {age}: {code} (n={count})")

# 2. Binary Encoding for GENDER
gender_mapping = {'Male': 1, 'Female': 0}
df_features['GENDER_ENCODED'] = df_features['GENDER'].map(gender_mapping)
print("\nGender Encoding:")
for gender, code in gender_mapping.items():
    count = (df_features['GENDER'] == gender).sum()
    print(f"  {gender}: {code} (n={count})")

# 3. One-Hot Encoding for AGE (for models that benefit from it)
age_dummies = pd.get_dummies(df_features['AGE'], prefix='AGE')
df_features = pd.concat([df_features, age_dummies], axis=1)

print("\nOne-Hot Encoded Age Columns:")
print([col for col in df_features.columns if col.startswith('AGE_')])

print(f"\nDataset shape after encoding: {df_features.shape}")


## 2.3 Standardize and Normalize Features

For clustering algorithms, we need to scale all features to the same range to prevent features with larger scales from dominating the distance calculations.


In [ ]:
print("=" * 80)
print("FEATURE STANDARDIZATION AND NORMALIZATION")
print("=" * 80)

# Select numerical features to scale (exclude original categorical columns and Timestamp)
exclude_cols = ['Timestamp', 'AGE', 'GENDER', 'AGE_ORIGINAL', 'GENDER_ORIGINAL']
numerical_features = [col for col in df_features.columns if col not in exclude_cols and not col.startswith('AGE_')]

print(f"\nNumber of numerical features to scale: {len(numerical_features)}")
print(f"Features: {numerical_features[:10]}...")  # Show first 10

# Create a copy for scaled features
df_scaled = df_features.copy()

# Method 1: StandardScaler (z-score normalization: mean=0, std=1)
# Best for algorithms that assume normally distributed data
scaler_standard = StandardScaler()
df_scaled[numerical_features] = scaler_standard.fit_transform(df_features[numerical_features])

print("\n" + "=" * 80)
print("STANDARDIZATION APPLIED (StandardScaler)")
print("=" * 80)
print("\nSample statistics after standardization:")
sample_features = ['WORK_LIFE_BALANCE_SCORE', 'PHYSICAL_HEALTH_SCORE', 
                   'MENTAL_HEALTH_SCORE', 'SOCIAL_HEALTH_SCORE']
for feature in sample_features:
    if feature in df_scaled.columns:
        print(f"{feature}:")
        print(f"  Mean: {df_scaled[feature].mean():.6f}")
        print(f"  Std: {df_scaled[feature].std():.6f}")
        print(f"  Min: {df_scaled[feature].min():.2f}")
        print(f"  Max: {df_scaled[feature].max():.2f}")

# Also create MinMax scaled version (0-1 range)
# Better for clustering algorithms like K-Means
df_minmax = df_features.copy()
scaler_minmax = MinMaxScaler()
df_minmax[numerical_features] = scaler_minmax.fit_transform(df_features[numerical_features])

print("\n" + "=" * 80)
print("MIN-MAX NORMALIZATION APPLIED (0-1 range)")
print("=" * 80)
print("\nSample statistics after min-max scaling:")
for feature in sample_features:
    if feature in df_minmax.columns:
        print(f"{feature}:")
        print(f"  Min: {df_minmax[feature].min():.6f}")
        print(f"  Max: {df_minmax[feature].max():.6f}")
        print(f"  Mean: {df_minmax[feature].mean():.4f}")

print(f"\nScaled dataset shape: {df_scaled.shape}")
print(f"MinMax scaled dataset shape: {df_minmax.shape}")


In [ ]:
# Visualize the effect of scaling
comparison_features = ['FRUITS_VEGGIES', 'DAILY_STRESS', 'WORK_LIFE_BALANCE_SCORE', 
                       'PHYSICAL_HEALTH_SCORE']

fig, axes = plt.subplots(3, 4, figsize=(20, 12))

for idx, feature in enumerate(comparison_features):
    # Original
    axes[0, idx].hist(df_features[feature], bins=30, color='steelblue', edgecolor='black', alpha=0.7)
    axes[0, idx].set_title(f'Original: {feature}', fontsize=10, fontweight='bold')
    axes[0, idx].set_ylabel('Frequency', fontsize=9)
    axes[0, idx].grid(alpha=0.3)
    
    # Standardized
    axes[1, idx].hist(df_scaled[feature], bins=30, color='coral', edgecolor='black', alpha=0.7)
    axes[1, idx].set_title(f'Standardized: {feature}', fontsize=10, fontweight='bold')
    axes[1, idx].set_ylabel('Frequency', fontsize=9)
    axes[1, idx].grid(alpha=0.3)
    
    # Min-Max Scaled
    axes[2, idx].hist(df_minmax[feature], bins=30, color='lightgreen', edgecolor='black', alpha=0.7)
    axes[2, idx].set_title(f'Min-Max: {feature}', fontsize=10, fontweight='bold')
    axes[2, idx].set_xlabel('Value', fontsize=9)
    axes[2, idx].set_ylabel('Frequency', fontsize=9)
    axes[2, idx].grid(alpha=0.3)

plt.suptitle('Comparison of Scaling Methods', fontsize=16, fontweight='bold', y=1.002)
plt.tight_layout()
plt.show()


## 2.4 Save Processed Datasets

We'll save multiple versions of the processed data for different use cases.


In [ ]:
print("=" * 80)
print("SAVING PROCESSED DATASETS")
print("=" * 80)

# Create data directory if it doesn't exist
data_dir = 'data'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    print(f"Created directory: {data_dir}/")
else:
    print(f"Directory already exists: {data_dir}/")

# Define file paths
files_to_save = {
    'Original with Features': (df_features, f'{data_dir}/wellbeing_with_features.csv'),
    'Standardized (Z-score)': (df_scaled, f'{data_dir}/wellbeing_standardized.csv'),
    'MinMax Normalized (0-1)': (df_minmax, f'{data_dir}/processed_wellbeing_data.csv')
}

# Save all versions
print("\nSaving files:")
for description, (dataframe, filepath) in files_to_save.items():
    dataframe.to_csv(filepath, index=False)
    file_size = os.path.getsize(filepath) / 1024  # Size in KB
    print(f"  ✓ {description}: {filepath}")
    print(f"    Shape: {dataframe.shape}, Size: {file_size:.2f} KB")

print("\n" + "=" * 80)
print("DATASET SUMMARY")
print("=" * 80)
print(f"\nOriginal columns: {len(df.columns)}")
print(f"New columns added: {len(df_features.columns) - len(df.columns)}")
print(f"Total columns in processed data: {len(df_features.columns)}")

print("\nNew features created:")
new_features = ['PHYSICAL_HEALTH_SCORE', 'MENTAL_HEALTH_SCORE', 'SOCIAL_HEALTH_SCORE',
                'AGE_ENCODED', 'GENDER_ENCODED']
for feature in new_features:
    if feature in df_features.columns:
        print(f"  ✓ {feature}")

print("\n" + "=" * 80)
print("FILES READY FOR CLUSTERING ANALYSIS!")
print("=" * 80)
print("\nRecommended file for clustering: data/processed_wellbeing_data.csv")
print("  - All features normalized to 0-1 range")
print("  - Composite health scores included")
print("  - Categorical variables encoded")
print("  - Ready for K-Means, DBSCAN, Hierarchical clustering")


In [ ]:
# Display final feature matrix summary
print("=" * 80)
print("FINAL FEATURE MATRIX FOR CLUSTERING")
print("=" * 80)

# Show sample of processed data
print("\nSample of processed data (first 5 rows, key features):")
key_columns = ['PHYSICAL_HEALTH_SCORE', 'MENTAL_HEALTH_SCORE', 'SOCIAL_HEALTH_SCORE',
               'WORK_LIFE_BALANCE_SCORE', 'AGE_ENCODED', 'GENDER_ENCODED']
print(df_minmax[key_columns].head())

print("\n" + "=" * 80)
print("Data types in processed dataset:")
print(df_minmax.dtypes.value_counts())

print("\n" + "=" * 80)
print("✓ Section 2: Feature Engineering & Preprocessing - COMPLETE")
print("=" * 80)
